In [229]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io

In [230]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'
database2= 'DB_TRAMITE_DOCUMENTARIO'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'

def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

def create_connection2():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database2};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn


In [231]:
def fetch_data():
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
SELECT 
    d.hoja_tramite,
    d.tipo_hoja,
    cdi.DESCRIPCION,
    d.INDICATIVO_OFICIO,
    p.RAZON_SOCIAL,
    d.FECHA_RECEPCION,
    d.ASUNTO,
    md.AUDIT_REC,
    do.SIGLAS AS SIGLAS_ORIGEN,
    do.DEPENDENCIA AS DEPENDENCIA_ORIGEN,
    dd.SIGLAS AS SIGLAS_DESTINO,
    dd.DEPENDENCIA AS DEPENDENCIA_DESTINO,
    ht.APELLIDOS_TRABAJADOR,
    ht.NOMBRES_TRABAJADOR,
    ed.ID_ESTADO_DOCUMENTO,
    ed.DESCRIPCION AS ESTADO_DOCUMENTO,
    md.AUDIT_TRAB_DER AS FECHA_DELEGADO,
    md.derivado,
    md.finalizado
FROM 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO md
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO d ON md.ID_DOCUMENTO = d.ID_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.dbo.PERSONA p ON d.ID_PERSONA = p.ID
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO cdi ON d.ID_CLASE_DOCUMENTO_INTERNO = cdi.ID_CLASE_DOCUMENTO_INTERNO
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.ESTADO_DOCUMENTO ed ON ed.ID_ESTADO_DOCUMENTO = d.ID_ESTADO_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA do ON do.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_ORIGEN 
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA dd ON dd.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_DESTINO
LEFT JOIN 
    DB_GENERAL.jcardenas.H_TRABAJADOR ht ON ht.CODIGO_TRABAJADOR = md.codigo_trabajador
WHERE
    (do.SIGLAS LIKE 'SG' OR dd.SIGLAS LIKE 'SG')
    AND
    md.AUDIT_TRAB_DER = (
        SELECT MAX(sub_md.AUDIT_TRAB_DER)
        FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO sub_md
        LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO sub_d ON sub_md.ID_DOCUMENTO = sub_d.ID_DOCUMENTO
        WHERE sub_d.hoja_tramite = d.hoja_tramite
    )
        """
        result = pd.read_sql(query, conn)
        result['estado']= np.where((result['ID_ESTADO_DOCUMENTO'] != 2) & (result['ID_ESTADO_DOCUMENTO'] != 10) & (result['derivado'] == 0) & (result['finalizado'] == 0),'Pendiente','Otros')
        today = pd.Timestamp.today()
        result['Días pendientes']= (today - result['FECHA_DELEGADO']).dt.days
        result['Hoja de trámite']=result['hoja_tramite']+result['tipo_hoja']
        result=result.sort_values(by='Días pendientes', ascending = False)


        result = result[result['estado']=='Pendiente']

    finally:
        conn.close()  # Cerrar la conexión al final
    return result





Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_15980\2307334269.py:81: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [ ]:
external_stylesheets = [dbc.themes.BOOTSTRAP,
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',  # Bootstrap CSS
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css']  # Bootstrap Icons
# Crear la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = dbc.Container([
    # Encabezado
    dbc.Row([
        dbc.Col([
            html.H1("var: Seguimiento del trámite", style={
                'color': '#0064AF',
                'fontSize': '28px',
                'textAlign': 'left',
                'fontWeight': 'bold',
                'fontFamily': 'Calibri'
            }),
            html.H2("Fuente: SGD", style={
                'color': '#0064AF',
                'fontSize': '12px',
                'textAlign': 'left',
                'fontFamily': 'Calibri'
            })
        ], width=12),
    ], style={'margin-top': '20px'}),

    # Input y filtros
    dbc.Row([
        dbc.Col([
            html.H6("# Hoja de tramite", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Input(
                id='hoja_tramite',
                type='text',
                style={'width': '100%', 'fontSize': '14px', 'display': 'inline-block','lineHeight': '10px','padding': '7px'}
            )
        ], width=12, md=6, lg=1, className='mb-2'),
        
        dbc.Col([
            html.H6("Días Pendientes", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Dropdown(
                id='dias_pendientes_dropdown',
                options=[
                    {'label': '1-7 días ✔️', 'value': '1-7'},
                    {'label': '8-15 días ⚠️', 'value': '8-15'},
                    {'label': '>15 días ❌', 'value': '>15'},
                    {'label': 'Todos', 'value': 'all'}
                ],
                value='all',
                style={'fontSize': '14px','height':'10px'}
            )
        ], width=12, md=6, lg=2, className='mb-2'),

        dbc.Col([
            html.H6("Fecha delegado", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.DatePickerRange(
                id='fecha_delegado',
                start_date=None,
                end_date=None,
                display_format='YYYY-MM-DD',  # Cambia el formato según sea necesario
                style={'fontSize': '14px'}
            )
        ], width=12, md=6, lg=3, className='mb-2'),

        dbc.Col([
            html.H6("Dependencia de origen", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Dropdown(
                id='dependencia_origen_dropdown',
                options=options,  # Actualiza con opciones reales
                value=None,
                placeholder="Seleccione una dependencia",
                style={'fontSize': '14px'}
            )
        ], width=12, md=6, lg=2, className='mb-2'),

        dbc.Col([
            html.H6("Dependencia de destino", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Dropdown(
                id='dependencia_destino_dropdown',
                options=options,  # Actualiza con opciones reales
                value=None,
                placeholder="Seleccione una dependencia",
                style={'fontSize': '14px'}
            )
        ], width=12, md=6, lg=2, className='mb-2'),

        dbc.Col([
            html.H6("Trabajador", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Dropdown(
                id='trabajador_dropdown',
                options=[{'label': 'Trabajador X', 'value': 'trabajador_x'}, {'label': 'Trabajador Y', 'value': 'trabajador_y'}],  # Actualiza con opciones reales
                value=None,
                placeholder="Seleccione un trabajador",
                style={'fontSize': '14px'}
            )
        ], width=12, md=6, lg=2, className='mb-2'),
    ], style={'margin-top': '20px'}),

    # Contenedor para el spinner y la tabla
    dbc.Row([
        dbc.Col([
            dbc.Spinner(
                id='spinner',
                color="primary",
                size="lg",
                children=html.Div(id='table_container'),
            )
        ], width=12)
    ], style={'margin-top': '20px'}),
    dcc.Store(id='data_store', data={})
], fluid=True)

@app.callback(
    [Output('table_container', 'children'),
     Output('spinner', 'fullscreen')],
    [Input('hoja_tramite', 'value'),
     Input('dias_pendientes_dropdown', 'value'),
     Input('fecha_delegado', 'start_date'),
     Input('fecha_delegado', 'end_date'),
     Input('dependencia_origen_dropdown', 'value'),
     Input('dependencia_destino_dropdown', 'value'),
     Input('trabajador_dropdown', 'value')]
)
def update_table(hoja_tramite_value, dias_pendientes_value, fecha_delegado_start, fecha_delegado_end, dependencia_origen_value, dependencia_destino_value, trabajador_value):
    # Cargar los datos solo una vez por solicitud
    df = fetch_data()
    options = [{'label': row['DEPENDENCIA_ORIGEN'], 'value': row['DEPENDENCIA_DESTINO']} for _, row in options.iterrows()]

    # Filtrar por número de hoja de trámite solo si el valor es proporcionado
    if hoja_tramite_value and hoja_tramite_value.strip():
        df_filtered = df[df['hoja_tramite'] == hoja_tramite_value]
    else:
        df_filtered = df.copy()

    # Aplicar el filtro de días pendientes
    if dias_pendientes_value and dias_pendientes_value != 'all':
        if dias_pendientes_value == '1-7':
            df_filtered = df_filtered[(df_filtered['Días pendientes'] >= 1) & (df_filtered['Días pendientes'] <= 7)]
        elif dias_pendientes_value == '8-15':
            df_filtered = df_filtered[(df_filtered['Días pendientes'] >= 8) & (df_filtered['Días pendientes'] <= 15)]
        elif dias_pendientes_value == '>15':
            df_filtered = df_filtered[df_filtered['Días pendientes'] > 15]

    # Filtrar por fecha delegado
    if fecha_delegado_start and fecha_delegado_end:
        df_filtered['fecha_delegado'] = pd.to_datetime(df_filtered['fecha_delegado'])
        df_filtered = df_filtered[(df_filtered['fecha_delegado'] >= fecha_delegado_start) & (df_filtered['fecha_delegado'] <= fecha_delegado_end)]

    # Filtrar por dependencia de origen
    if dependencia_origen_value:
        df_filtered = df_filtered[df_filtered['DEPENDENCIA_ORIGEN'] == dependencia_origen_value]

    # Filtrar por dependencia de destino
    if dependencia_destino_value:
        df_filtered = df_filtered[df_filtered['DEPENDENCIA_DESTINO'] == dependencia_destino_value]

    # Filtrar por trabajador
    if trabajador_value:
        df_filtered = df_filtered[df_filtered['NOMBRE_TRABAJADOR'] == trabajador_value]

    # Verificar si el DataFrame filtrado está vacío
    if not df_filtered.empty:
        # Añadir íconos a la columna "Días pendientes"
        df_filtered['Días pendientes'] = df_filtered['Días pendientes'].apply(
            lambda days: f'{days}  ✔️' if 1 <= days <= 7 else
                         f'{days}  ⚠️' if 8 <= days <= 15 else
                         f'{days}  ❌'
        )

        # Crear la DataTable
        table = dash_table.DataTable(
            id='data_table',
            columns=[{"name": i, "id": i} for i in ['Hoja de trámite', 'DESCRIPCION', 'INDICATIVO_OFICIO', 'RAZON_SOCIAL', 'ASUNTO', 'DEPENDENCIA_ORIGEN', 'DEPENDENCIA_DESTINO', 'NOMBRE_TRABAJADOR', 'Días pendientes']],
            data=df_filtered.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'fontFamily': 'Calibri',
                'padding': '5px',
                'height': 'auto',
                'maxWidth': '120px',
                'whiteSpace': 'normal',
                'color': '#606060',
                'fontSize': '14px'
            },
            style_header={
                'backgroundColor': '#0064AF',
                'fontWeight': 'bold',
                'color': 'white'
            },
            style_cell_conditional=[
                {'if': {'column_id': 'Hoja de trámite'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '150px', 'textAlign': 'center'},
                {'if': {'column_id': 'DESCRIPCION'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '200px', 'textAlign': 'center'},
                {'if': {'column_id': 'INDICATIVO_OFICIO'}, 'minWidth': '130px', 'width': '130px', 'maxWidth': '250px'},
                {'if': {'column_id': 'RAZON_SOCIAL'}, 'minWidth': '120px', 'width': '120px', 'maxWidth': '150px', 'textAlign': 'center'},
                {'if': {'column_id': 'ASUNTO'}, 'minWidth': '230px', 'width': '230px', 'maxWidth': '250px'},
                {'if': {'column_id': 'DEPENDENCIA_ORIGEN'}, 'minWidth': '120px', 'width': '120px', 'maxWidth': '150px', 'textAlign': 'center'},
                {'if': {'column_id': 'DEPENDENCIA_DESTINO'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '150px'},
                {'if': {'column_id': 'NOMBRE_TRABAJADOR'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px'},
                {'if': {'column_id': 'Días pendientes'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px', 'textAlign': 'center'},
            ],
            style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(244, 250, 253)'}],
            page_size=8  # Número de filas por página
        )
        return table, False
    else:
        # Devuelve un mensaje indicando que no hay datos y que el spinner no esté en pantalla completa
        return html.Div("No se encontraron resultados con los filtros aplicados", style={'color': '#606060', 'fontWeight': 'bold'}), False
if __name__ == '__main__':
    app.run_server(debug=True)

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_15980\2307334269.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.
Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_15980\2307334269.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_15980\2307334269.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

